In [1]:
#RUN THIS
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
#RUN THIS
!pip3 install transformers

     |████████████████████████████████| 368kB 5.0MB/s 
     |████████████████████████████████| 860kB 46.8MB/s 
     |████████████████████████████████| 645kB 38.9MB/s 
     |████████████████████████████████| 1.0MB 43.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=2df30418aa637d60af6f87bb61f3bc98869b9ccc6e2a6fc15ecc08e27c9bc490
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [3]:

import torch
import numpy as np
import pandas as pd
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig

In [0]:


def read_data(filepath):
	print("reading ",filepath)
	genes = []
	labels = []
	df = pd.read_csv(filepath,usecols=[1,2],sep="\t",skiprows=1,header=None)
	print(df.head())
	for entry in df.itertuples():
		kmer_list = [kmer.strip("\'") for kmer in entry[1][1:-1].split(", ")]
		label_list = list(map(float, entry[2][1:-1].split(", ")))
		genes.append(kmer_list)
		labels.append(label_list)
	return genes, labels
	

def tokenize_samples(genes):
  k= len(genes[0][0])
  if k==2:
    kmer_filepath = '/content/drive/My Drive/fourmersXL.txt'
  elif k==6:
    kmer_filepath = '/content/drive/My Drive/hexamersXL.txt'

  tokenizer=TransfoXLTokenizer(vocab_file=kmer_filepath)
  print("TOKENIZER LENGTH", len(tokenizer))
  seq_ids = [tokenizer.convert_tokens_to_ids(gene) for gene in genes]
  return seq_ids


In [0]:
#RUN THIS
from transformers import TransfoXLTokenizer, TransfoXLModel, TransfoXLConfig
class Model(torch.nn.Module):
  def __init__(self):
    super(Model, self).__init__()
    self.config = TransfoXLConfig(vocab_size_or_config_json_file='/content/drive/My Drive/classification_XL_configuration.json')
    self.config.vocab_size=204098
    self.model = TransfoXLModel(self.config)
    self.out_layer = torch.nn.Linear(self.model.d_model, 2)
  def forward(self, input_ids, mems=None):
    hidden_states, mems = self.model(input_ids, mems)
    preds = self.out_layer(hidden_states[0]).squeeze(0)
    return preds, mems


In [0]:
#RUN THIS
window_size = 1012
device = torch.device('cuda')
#building model
model = Model()
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
from torch.nn import CrossEntropyLoss
class_weights = torch.tensor(np.array([1.0, 20.0])).float().cuda()
loss_func = CrossEntropyLoss(weight=class_weights)



In [7]:
#RUN THIS
genes,labels = read_data('/content/drive/My Drive/all_samples_6-mer_val.txt')
gene_ids = tokenize_samples(genes)
print(gene_ids[0])
print(len(gene_ids))
print("Finished making data")

reading  /content/drive/My Drive/all_samples_6-mer_val.txt
                                                   1                                                  2
0  ['CGTCAC', 'CCTTGT', 'TTTCCG', 'GGCTGT', 'GAGT...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
1  ['CGCGCA', 'GCATGC', 'CCGCCA', 'GCGCCC', 'CGCC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
2  ['TTAGTG', 'CAGGGG', 'GCGGGC', 'CCGGGA', 'CGCA...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
3  ['GTGCGA', 'CCGGGG', 'GGCGGT', 'TGGGGT', 'TCAC...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
4  ['ACCAGT', 'ACCAGC', 'CCCCAC', 'CTTCCG', 'TCTG...  [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...
TOKENIZER LENGTH 4098
[1939, 1456, 2713, 3952, 3304, 1359, 3671, 876, 1344, 1011, 3415, 2387, 3382, 2027, 2017, 3826, 977, 3953, 3479, 887, 1879, 2443, 1171, 759, 3893, 2322, 18, 1244, 3030, 793, 2844, 1250, 1010, 1409, 3483, 3383, 856, 1463, 1214, 3861, 2712, 2315, 2737, 3320, 3288, 1335, 1334, 3382, 3385, 3385, 34

In [0]:
#RUN THIS
from torch.nn import Softmax
softmax = Softmax(dim=1)
#how often to print stuff
genes_per_cycle=100
save_path = "/content/drive/My Drive/xl_classification_6mer_sd_ftMKIII.pt"

def save_weights():
  print("Saving weights")
  torch.save(model.state_dict(), save_path)

def get_topk_predicted_ss(tensor_preds,k):
  normalized_preds = softmax(tensor_preds)
  np_preds = normalized_preds.detach().cpu().numpy()
  #take only probability of ss and argsort by that
  prob_of_ss_arr=np.transpose(np_preds)[1]
  topk_results = np.argsort(prob_of_ss_arr)[-k:]
  return topk_results




#at the end of every "cycle"
def print_stats(np_preds,np_labels):
  predicted_ss = []
  for i in range(len(np_preds)):
    if(np_preds[i][1]>np_preds[i][0]):
      predicted_ss.append(i)
  labeled_ss = np.nonzero(np_labels)[0]
  correctly_predicted = np.intersect1d(predicted_ss,labeled_ss)
  num_ss_in_label = len(labeled_ss)
  print("Number of SS in Target: {}; Number of SS Predicted: {}".format(num_ss_in_label,len(predicted_ss)))
  if(len(predicted_ss)>0):
    print("Precision: ", len(correctly_predicted)/len(predicted_ss))
  else:
    print("None Predicted")
  print("Recall: ", len(correctly_predicted)/num_ss_in_label)
  topk_preds = get_topk_predicted_ss(torch.tensor(np_preds),num_ss_in_label)
  topk_accuracy = len(np.intersect1d(topk_preds,labeled_ss))/num_ss_in_label
  print("Top-K: ", topk_accuracy)

def run_epoch(genes,labels):
  epoch_loss=0
  cycle_loss = 0
  num_samples=len(genes)
  for i in range(num_samples):
    mems = None
    gene_loss = 0
    curr_gene = genes[i]
    curr_labels = labels[i]
    list_preds=[]
    #looping over all windows
    for w in range(0, len(curr_gene), window_size):
      toks = curr_gene[w:w+window_size]
      #ignore tiny windows(leads to dimensionality issues)
      if(len(toks)<2):
        continue
      window_input_ids = torch.tensor(toks).unsqueeze(0).cuda()
      window_labels = torch.tensor(curr_labels[w:w+window_size]).long().cuda()
      window_preds, mems = model(window_input_ids, mems)
      l = loss_func(window_preds, window_labels)
      gene_loss += l.item()
      #adds loss to storage
      l.backward()
      list_preds = list_preds+list(window_preds.detach().cpu().numpy())
    optimizer.step()
    optimizer.zero_grad()
    cycle_loss += gene_loss
    if i > 0 and i % genes_per_cycle == 0:
      np_preds=np.array(list_preds)
      print("\nAverage Loss Per Gene({}): ".format(i) + str(cycle_loss / genes_per_cycle))
      print_stats(np_preds,np.array(curr_labels))
      epoch_loss+=cycle_loss
      cycle_loss = 0
    if i > 0 and i % 1000 == 0:
      save_weights()
  #For overfitting
  #print("Loss on Epoch: "+str(cycle_loss/num_samples))
  #print_stats(np.array(list_preds),np.array(curr_labels))
  print("\nLoss on Epoch: "+str(epoch_loss/num_samples))
  save_weights()




In [0]:
#try changing class weights after running for a bit
class_weights = torch.tensor(np.array([1.0, 165.0])).float().cuda()
loss_func = CrossEntropyLoss(weight=class_weights)

In [9]:
#RUN THIS
#TURN ON WEIGHT SAVING
f="/content/drive/My Drive/xl_classification_6mer_sd_ftMKIII.pt"
model.load_state_dict(torch.load(f))
model.train()
for i in range(10):
  run_epoch(gene_ids,labels)


Average Loss Per Gene(100): 0.8172108840290457
Number of SS in Target: 45; Number of SS Predicted: 106
Precision:  0.11320754716981132
Recall:  0.26666666666666666
Top-K:  0.17777777777777778

Average Loss Per Gene(200): 0.8403802775591612
Number of SS in Target: 80; Number of SS Predicted: 344
Precision:  0.09011627906976744
Recall:  0.3875
Top-K:  0.1625

Average Loss Per Gene(300): 0.8427815910056233
Number of SS in Target: 2; Number of SS Predicted: 105
Precision:  0.009523809523809525
Recall:  0.5
Top-K:  0.0

Average Loss Per Gene(400): 0.9672143548168242
Number of SS in Target: 22; Number of SS Predicted: 65
Precision:  0.09230769230769231
Recall:  0.2727272727272727
Top-K:  0.22727272727272727

Average Loss Per Gene(500): 1.0259462811332196
Number of SS in Target: 16; Number of SS Predicted: 103
Precision:  0.10679611650485436
Recall:  0.6875
Top-K:  0.3125

Average Loss Per Gene(600): 1.0537828528508544
Number of SS in Target: 30; Number of SS Predicted: 417
Precision:  0.057

In [0]:
#Overfitting to small training sample
training_genes = gene_ids[:10]
training_labels = labels[:10]
testing_genes=gene_ids[10:11]
testing_labels=labels[10:11]
for i in range(0,30):
  model.train()
  run_epoch(training_genes,training_labels)



Loss on Epoch: 2.2800252810120583
Number of SS in Target: 60; Number of SS Predicted: 57
Precision:  0.017543859649122806
Recall:  0.016666666666666666
Top-K:  0.03333333333333333
Loss on Epoch: 2.0711057245731355
Number of SS in Target: 60; Number of SS Predicted: 1194
Precision:  0.020100502512562814
Recall:  0.4
Top-K:  0.06666666666666667
Loss on Epoch: 1.671315062046051
Number of SS in Target: 60; Number of SS Predicted: 608
Precision:  0.029605263157894735
Recall:  0.3
Top-K:  0.016666666666666666
Loss on Epoch: 1.463560402393341
Number of SS in Target: 60; Number of SS Predicted: 561
Precision:  0.049910873440285206
Recall:  0.4666666666666667
Top-K:  0.08333333333333333
Loss on Epoch: 1.329523818194866
Number of SS in Target: 60; Number of SS Predicted: 659
Precision:  0.05614567526555387
Recall:  0.6166666666666667
Top-K:  0.11666666666666667
Loss on Epoch: 1.1295791864395142
Number of SS in Target: 60; Number of SS Predicted: 537
Precision:  0.07262569832402235
Recall:  0.65
